<a href="https://colab.research.google.com/github/donnkh/final/blob/main/asian_premium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import math 
import numpy as np
def binom_coef(n,k) :
  a = math.factorial(n)
  b= math.factorial(k)
  c = math.factorial(n-k)
  return int(a/(b*c))
  
a =format(1,'b')
print(a)
a.rjust(2,'0')

1


'01'

In [50]:
S0=100
K=100
N=30
M=1000
sig=0.2
T=N/365
dt=T/N
u= np.exp(sig*np.sqrt(dt))
print(u)
d = 1/ u
r = 1
q1 = (np.exp(r*dt) - d) / (u - d)
q2 = ( u - np.exp(r*dt) ) / (u - d)
print(q1)
print(q2)

1.0105234646788943
0.6284159102081341
0.3715840897918658


In [51]:
#0부터 1023(2^10 - 1 )까지 인덱스로 시나리오를 만든다. 이때 1일차엔 2^1개, 2일차엔 2^2개... N일차엔 2^N개의 시나리오가 들어간다.
#그 시나리오마다 각자의 주가경로를 구한다. 그것을 리스트로 저장한다. 이후 마지막기 평균을 구한다. list_n = [u,ud,udd,uddd,,,,] -> avg_n = u+ud+udd+,,,, / 기간
#마지막기 평균만 있으면 된다. 왜? 마지막기 평균을 가지고 옵션의 페이오프를 구한뒤 그것을 현가화 시키는 과정이 프라이싱이기 때문.

In [54]:
senario_list =[]
for i in range(2**N) :
  senario_number = (format(i,'b'))
  senario_list.append(senario_number.rjust(N,'0'))
#s_l에는 0부터 2^^N-1이 이진법으로 들어가있음 ex)001011011 여기서 0은 up, 1은 down으로 생각


KeyboardInterrupt: ignored

In [48]:
print(senario_list[40])

00000000000000101000


In [37]:
def up_down(senario) :
  uside = str(senario).count('0')
  #print(uside)
  dside = str(senario).count('1')
  #print(dside)

  return S0 * (u**uside) * (d**dside)
#0과 1의 개수를 세서 그만큼 u과 d으로 바꾸어 그 날의 주가를 구함

In [38]:
def day_slice(senario) :
  day_list =[]
  for i in range(len(senario)) :
    day_list.append(senario[:i+1])

  return day_list
#시나리오를 1일차,2일차,,,N일차까지 나눈 리스트를 반환

In [39]:
def day_avg(day_list ) : # 1,2,,,n일차까지의 주가가 들어있는 리스트에서 주가의 합을 n으로 나눈 산술평균을 반환
  return sum(day_list) / N


In [40]:
#def day_avg_list(day_list) :
#  day_avg_list = []
#  for i in range(len(day_list)) :
#    day_avg_list.append(day_avg(day_list,i))
  
#  return day_avg_list
      

In [41]:
def go_past(q1_factor,q2_factor) :
  return q1*q1_factor + q2*q2_factor


In [42]:
def asian_payoff(S_avg,K) :
  return np.max(S_avg-K , 0)

In [43]:
def go_yesterday(value_list) :
  result_list =[]
  for i in range(int(len(value_list)/2)) :
    # 길이가 2의 3승(8)이면 인덱스가 0123 4개로 주어짐. 2*0이랑 2*0+1 / 2*1이랑 2*1+1이랑 /
    result_list.append(go_past(value_list[i*2],value_list[i*2+1]))

  return result_list 


In [44]:
def binomial_pricing(value_list) :
  for i in range(N) :
    value_list = go_yesterday(value_list)
  return value_list


In [45]:
def asian_premium(senario_list) :
  value_list =[]

  for i in range(len(senario_list)) :
    day_sliced_list = day_slice(senario_list[i])
    for j in range(len(day_sliced_list)) :
      day_sliced_list[j] = up_down(day_sliced_list[j])
    value_list.append(day_avg(day_sliced_list))
  
  for i in range(len(value_list)) :
    value_list[i] = asian_payoff(value_list[i],K)
  

  value_list = binomial_pricing(value_list)
  
  return value_list


    


In [46]:
print(asian_premium(senario_list))

[2.9313329004957183]
